# Spatial Duck

```python
install spatial;
load spatial;
select * from 'https://whatever/shapefile/on/the/internet.geojson';

```


References
* [DuckDB: The indispensable geospatial tool you didn’t know you were missing](https://medium.com/radiant-earth-insights/duckdb-the-indispensable-geospatial-tool-you-didnt-know-you-were-missing-5fe11c5633e5)
* https://duckdb.org/2023/04/28/spatial.html
* https://developmentseed.org/lonboard/latest/ecosystem/duckdb/
* https://github.com/duckdb/duckdb_spatial/blob/main/docs/example.md



Need to try

```sql
SELECT DISTINCT ST_GeometryType(ST_GeomFromWKB(geometry)) AS geom_type FROM 'PathTo\\*.parquet' LIMIT 5;
```

from https://stackoverflow.com/questions/77605626/duckdb-st-geometrytypeblob-add-explicit-type-casts

In [3]:
import duckdb

In [4]:
duckdb.install_extension("spatial")
duckdb.install_extension("httpfs")

geocon = duckdb.connect()
geocon.execute("INSTALL spatial;")
geocon.execute("LOAD spatial;")
geocon.execute("INSTALL httpfs;")
geocon.execute("LOAD httpfs;")

In [5]:
jsonurl = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json"
shpurl = "https://github.com/nvkelso/natural-earth-vector/raw/master/110m_physical/ne_110m_land.shp"

In [7]:
geocon.execute("CREATE TABLE geojson_data (geometry GEOMETRY,properties JSON);")

In [8]:
# Load the GeoJSON data using ST_Read function
# geocon.execute("DROP TABLE data")
geocon.execute(f"CREATE TABLE data AS SELECT * FROM ST_Read('{jsonurl}');")
# geocon.execute(f"INSERT INTO geojson_data (geometry, properties) SELECT ST_Read(geojson), properties FROM '{jsonurl}';")

In [9]:
r1 = geocon.execute("DESCRIBE SELECT * FROM data").fetchdf()
print(list(r1['column_name']))

['id', 'name', 'density', 'geom']


In [10]:
r2 = geocon.execute("SELECT * FROM data").fetchdf()
r2.head()

,id,name,density,geom
0,01,Alabama,94.650,"[2, 4, 0, 0, 0, 0, 0, 0, 54, 241, 176, 194, 65..."
1,02,Alaska,1.264,"[5, 4, 0, 0, 0, 0, 0, 0, 169, 231, 60, 195, 11..."
2,04,Arizona,57.050,"[2, 4, 0, 0, 0, 0, 0, 0, 98, 161, 229, 194, 45..."
3,05,Arkansas,56.430,"[2, 4, 0, 0, 0, 0, 0, 0, 133, 59, 189, 194, 37..."
4,06,California,241.700,"[2, 4, 0, 0, 0, 0, 0, 0, 85, 210, 248, 194, 11..."


In [11]:
geor2 = geocon.execute("SELECT * FROM 'https://github.com/nvkelso/natural-earth-vector/raw/master/110m_physical/ne_110m_land.shp';").fetchdf()

geor2.head(10)

,featurecla,scalerank,min_zoom,geom
0,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 127, 148, 132, 194, 4..."
1,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 129, 182, 35, 195, 18..."
2,Land,1,0.0,"[2, 4, 0, 0, 0, 0, 0, 0, 52, 168, 88, 194, 7, ..."
3,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 85, 62, 245, 194, 121..."
4,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 247, 144, 254, 194, 2..."
5,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 85, 169, 204, 194, 22..."
6,Land,1,1.0,"[2, 4, 0, 0, 0, 0, 0, 0, 119, 6, 150, 194, 248..."
7,Land,0,0.0,"[2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 52, 195, 1, 0, ..."
8,Land,0,0.0,"[2, 4, 0, 0, 0, 0, 0, 0, 56, 83, 149, 194, 132..."
9,Land,0,0.5,"[2, 4, 0, 0, 0, 0, 0, 0, 205, 204, 116, 194, 5..."


In [13]:
import duckdb
import geopandas as gpd

# Load GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(jsonurl)

In [14]:
gdf.head()

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,04,Arizona,57.050,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,05,Arkansas,56.430,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,06,California,241.700,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [16]:
# Connect to DuckDB (in-memory or to a file)
# conn = duckdb.connect(database=':memory:')

# Load GeoDataFrame into DuckDB
# geocon.register('geo_table', gdf)

# Query the table in DuckDB
# result = geocon.execute('SELECT * FROM geo_table').fetchall()

# Display the result
# for row in result:
#     print(row)